## 구글 클라우드 비전 api를 사용해 유튜브 썸네일을 이미지 캡셔닝, OCR.

In [ ]:
%pip install google-cloud-vision

In [ ]:
%pip install python-dotenv

### 구글 클라우드에서 활성화한 api키를 .env파일에 저장

In [ ]:
# 필요한 라이브러리 불러오기
from dotenv import load_dotenv
import os

# .env 파일에 저장된 환경 변수 로드 (기본적으로 현재 작업 디렉터리의 .env 파일을 찾습니다)
load_dotenv()

# 환경 변수 호출하기
cloud_vis_api = os.getenv("CLOUD_VIS_API_2")

# 확인용 출력
print(cloud_vis_api)

### csv 파일에서 URL을 읽을 때

In [ ]:
import os
import pandas as pd
from datetime import datetime
from google.cloud import vision


# 클라이언트 생성
client = vision.ImageAnnotatorClient(client_options={"api_key": cloud_vis_api})

# 신뢰도 임계값 및 불용어 설정
confidence_threshold = 0.5
generic_stopwords = {"Internet", "Image", "Photo caption"}
# CSV 파일 로드
csv_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\videos_by_keywords_최종.csv"
df = pd.read_csv(csv_path, low_memory=False)

# [uploadDate]를 datetime으로 변환
df['uploadDate'] = pd.to_datetime(df['uploadDate']).dt.tz_localize(None)
# 최종 결과 CSV 경로 지정 (필터링된 데이터용 별도 저장)
output_csv_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\videos_with_captions_0312_0322.csv"
# 날짜 필터링 적용 (예: 2025-02-19 ~ 2025-02-22)
start_date = datetime(2025, 3, 11, 23, 59, 59)  # 2025, 2, 19, 0, 0, 0
end_date = datetime(2025, 3, 22, 23, 59, 59)  # 2025, 2, 22, 23, 59, 59
df_filtered = df[(df['uploadDate'] >= start_date) & (df['uploadDate'] <= end_date)].copy()

# 결과 저장 열 추가
df_filtered['img_key_word'] = ""
df_filtered['img_text'] = ""
print(f"총 {len(df_filtered)}개의 영상이 지정된 날짜 기간에 포함되었습니다.")
# 배치 처리 설정 (배치 크기 설정, 16개가 한번에 할 수 있는 최대)
batch_size = 16
image_requests = []
indices = []

# 각 이미지에 대한 AnnotateImageRequest 생성
for idx, row in df_filtered.iterrows():
    url = row['thumbnailURL']
    # Vision API가 웹 URL을 직접 처리할 수 있도록 이미지 소스 지정
    image = vision.Image(source=vision.ImageSource(image_uri=url))
    request = vision.AnnotateImageRequest(
        image=image,
        features=[
            vision.Feature(type_=vision.Feature.Type.LABEL_DETECTION),
            vision.Feature(type_=vision.Feature.Type.OBJECT_LOCALIZATION),
            vision.Feature(type_=vision.Feature.Type.WEB_DETECTION),
            vision.Feature(type_=vision.Feature.Type.TEXT_DETECTION)
        ]
    )
    image_requests.append(request)
    indices.append(idx)

# 배치별로 요청 수행
for start in range(0, len(image_requests), batch_size):
    batch_requests = image_requests[start:start+batch_size]
    batch_indices = indices[start:start+batch_size]
    
    response = client.batch_annotate_images(requests=batch_requests)
    responses = response.responses

    for idx, res in zip(batch_indices, responses):
        # 라벨, 객체, 웹 엔터티 처리
        label_descriptions = [label.description for label in res.label_annotations if label.score >= confidence_threshold] if res.label_annotations else []
        object_names = [obj.name for obj in res.localized_object_annotations if obj.score >= confidence_threshold] if res.localized_object_annotations else []
        web_descriptions = [entity.description for entity in res.web_detection.web_entities if entity.description] if res.web_detection and res.web_detection.web_entities else []
        
        combined_keywords = []
        for keyword in label_descriptions + object_names + web_descriptions:
            if keyword in generic_stopwords or keyword.lower() in map(str.lower, combined_keywords):
                continue
            combined_keywords.append(keyword)
        keywords_str = ", ".join(combined_keywords)
        
        # OCR 텍스트 처리
        detected_text = ""
        if res.text_annotations:
            detected_text = ' '.join(res.text_annotations[0].description.split())
        
        df_filtered.at[idx, 'img_key_word'] = keywords_str
        df_filtered.at[idx, 'img_text'] = detected_text
    
    # 배치 처리 후 불필요한 "Unnamed" 열 제거 후 CSV 저장
    df_filtered = df_filtered.drop([col for col in df_filtered.columns if col.startswith("Unnamed")], axis=1)
    df_filtered.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    print(f"{min(start+batch_size, len(image_requests))}/{len(image_requests)} 이미지 배치 처리 후 CSV 저장 완료.")

print("모든 이미지 배치 처리가 완료되었습니다.")

### DB에서 URL을 읽을 때

In [ ]:
from dotenv import load_dotenv
import os

# .env 파일의 환경 변수 로드
load_dotenv()

# 환경 변수에서 DB 암호 가져오기 (여기서는 변수명이 '1P_DB_PW'라고 가정)
db_password = os.getenv('1P_DB_PW')
print(db_password)

In [ ]:
import os
import pandas as pd
from datetime import datetime
from google.cloud import vision
import mysql.connector  # MySQL 접속 라이브러리

# DB 접속 설정
db_config = {
    'host': '121.128.172.79',
    'user': 'user3',
    'password': db_password,
    'database': 'yttest_db',
    'port': 3306
}

# DB 연결 생성
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# 클라이언트 생성 (API 키는 cloud_vis_api 변수에 저장되어 있어야 함)
client = vision.ImageAnnotatorClient(client_options={"api_key": cloud_vis_api})

# 신뢰도 임계값 및 불용어 설정
confidence_threshold = 0.5
generic_stopwords = {"Internet", "Image", "Photo caption"}

# CSV 파일 로드 (필요한 경우 DB에서 가져오거나 CSV를 계속 사용할 수 있음)
csv_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\videos_by_keywords_최종.csv"
df = pd.read_csv(csv_path, low_memory=False)

# [uploadDate]를 datetime으로 변환
df['uploadDate'] = pd.to_datetime(df['uploadDate']).dt.tz_localize(None)
# 날짜 필터링 적용 (예: 2025-03-11 ~ 2025-03-22)
start_date = datetime(2025, 3, 11, 23, 59, 59)
end_date = datetime(2025, 3, 22, 23, 59, 59)
df_filtered = df[(df['uploadDate'] >= start_date) & (df['uploadDate'] <= end_date)].copy()

print(f"총 {len(df_filtered)}개의 영상이 지정된 날짜 기간에 포함되었습니다.")

# 배치 처리 설정 (한 번에 처리할 이미지 개수: 16)
batch_size = 16
image_requests = []
indices = []

# 각 이미지에 대한 AnnotateImageRequest 생성
for idx, row in df_filtered.iterrows():
    url = row['thumbnailURL']
    image = vision.Image(source=vision.ImageSource(image_uri=url))
    request = vision.AnnotateImageRequest(
        image=image,
        features=[
            vision.Feature(type_=vision.Feature.Type.LABEL_DETECTION),
            vision.Feature(type_=vision.Feature.Type.OBJECT_LOCALIZATION),
            vision.Feature(type_=vision.Feature.Type.WEB_DETECTION),
            vision.Feature(type_=vision.Feature.Type.TEXT_DETECTION)
        ]
    )
    image_requests.append(request)
    indices.append(idx)

# 배치별 요청 및 DB 업데이트 수행
for start in range(0, len(image_requests), batch_size):
    batch_requests = image_requests[start:start+batch_size]
    batch_indices = indices[start:start+batch_size]
    
    response = client.batch_annotate_images(requests=batch_requests)
    responses = response.responses

    for idx, res in zip(batch_indices, responses):
        # 라벨, 객체, 웹 엔터티 처리
        label_descriptions = [label.description for label in res.label_annotations if label.score >= confidence_threshold] if res.label_annotations else []
        object_names = [obj.name for obj in res.localized_object_annotations if obj.score >= confidence_threshold] if res.localized_object_annotations else []
        web_descriptions = [entity.description for entity in res.web_detection.web_entities if entity.description] if res.web_detection and res.web_detection.web_entities else []
        
        combined_keywords = []
        for keyword in label_descriptions + object_names + web_descriptions:
            if keyword in generic_stopwords or keyword.lower() in map(str.lower, combined_keywords):
                continue
            combined_keywords.append(keyword)
        keywords_str = ", ".join(combined_keywords)
        
        # OCR 텍스트 처리
        detected_text = ""
        if res.text_annotations:
            detected_text = ' '.join(res.text_annotations[0].description.split())
        
        # thumbnailURL을 키로 하여 DB 업데이트 (thumbnail 테이블의 imgKw, imgText 컬럼)
        thumbnail_url = df_filtered.at[idx, 'thumbnailURL']
        update_query = """
            UPDATE thumbnail
            SET imgKw = %s,
                imgText = %s
            WHERE thumbnailURL = %s
        """
        cursor.execute(update_query, (keywords_str, detected_text, thumbnail_url))
    
    conn.commit()
    print(f"{min(start+batch_size, len(image_requests))}/{len(image_requests)} 이미지 배치 처리 후 DB 업데이트 완료.")

cursor.close()
conn.close()
print("모든 이미지 배치 처리가 완료되었습니다.")